## Imports

In [ ]:
import datetime as dt
import yfinance as yf
from pandas_datareader import data as pdr
import pandas as pd
import requests
import bs4 as bs
import pickle
import os
yf.pdr_override()

## Scrapping and saving data (csv)

In [ ]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.replace('.', '-')
        ticker = ticker[:-1]
        tickers.append(ticker)
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
    return tickers

def get_data_from_yahoo(reload_sp500=False):    
    if reload_sp500:
        tickers = guardar_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
            if not os.path.exists('financial_data'):
                os.makedirs('financial_data')
    for ticker in tickers:     
                    if not os.path.exists('financial_data/{}.csv'.format(ticker)):
                        tick = yf.Ticker(ticker).info
                        df = pd.DataFrame(list(tick.items())) 
                        df.to_csv('financial_data/{}.csv'.format(ticker))
                        df.rename(columns = {0:'Data', 1:ticker}, inplace=True)
                        print(ticker)
                    else:
                         print('Already have {}'.format(ticker))

save_sp500_tickers()
get_data_from_yahoo()

## Compile data (WiP)